## Finding the most profitable towns in a CCAA given a max price

In [32]:
import pandas as pd
import math
from bs4 import BeautifulSoup
import requests
import nums_from_string as nfs
import numpy as np
from re import search
from random import randint
from time import sleep
import openpyxl
import xlrd
import lxml
from datetime import datetime, timedelta
from collections import Counter


#### Importing geo data from csv file

In [2]:
geo_data = pd.read_csv('/Users/ignaciolorenzoqueralt/Documents/Ironhack/Final Project/data/province-town-n_props/geo-data_2021.10.23.csv')

In [3]:
geo_data.head(3)

,Unnamed: 0,town,n_properties,province,ccaa
0,0,alegria___dulantzi,26,alava,pais_vasco
1,1,amurrio,6,alava,pais_vasco
2,2,anana,2,alava,pais_vasco


#### Get input from user

In [4]:
ccaa_lst = geo_data['ccaa'].unique().tolist()
ccaa = input("ccaa: ")
while ccaa not in ccaa_lst:
    ccaa = input("There was no match between your input and our ccaa, try again: ")

ccaa: cataluña


In [5]:
num_properties_per_town = ""
while num_properties_per_town == "":
    try:
        num_properties_per_town = int(input("Minimum number of properties per town: "))
    except: 
        num_properties_per_town = ""

Minimum number of properties per town: 500


In [6]:
filtered_df = geo_data[(geo_data['ccaa'] == ccaa) & (geo_data['n_properties'] > num_properties_per_town)]

In [7]:
filtered_df['ccaa'].unique()

array(['cataluña'], dtype=object)

In [8]:
filtered_df.shape

(39, 5)

In [9]:
max_price = ""
while max_price == "":
    try:
        max_price = int(input("maximum price: "))
    except: 
        max_price = ""

maximum price: 90000


#### List of towns that we need to extract data from

In [10]:
towns = filtered_df[filtered_df['ccaa'] == ccaa].town.to_list()

In [11]:
towns

['badalona',
 'barcelona',
 'castelldefels',
 'cornella_de_llobregat',
 'cubelles',
 'gava',
 'granollers',
 'hospitalet_de_llobregat',
 'manresa',
 'mataro',
 'mollet_del_valles',
 'pineda_de_mar',
 'rubi',
 'sabadell',
 'sant_boi_de_llobregat',
 'sant_cugat_del_valles',
 'sant_pere_de_ribes',
 'santa_coloma_de_gramenet',
 'sitges',
 'terrassa',
 'tordera',
 'vic',
 'viladecans',
 'vilanova_i_la_geltru',
 'alcanar',
 'ametlla_de_mar',
 'calafell',
 'cambrils',
 'cunit',
 'mont_roig_del_camp',
 'reus',
 'salou',
 'sant_carles_de_la_rapita',
 'santa_oliva',
 'segur_de_calafell',
 'tarragona',
 'tortosa',
 'vendrell',
 'vila_seca']

Hay que hacer un for loop que para cada town saque el df y le haga un append a el global.

#### Extracting data from each town

In [12]:
property = "viviendas"
municipio = "hospitalet_de_llobregat"
ascensor = "-ascensor"

habitaciones = "hab="+"1"
baños = "&"+"ban="+"1"
maximum_price = "pmax="+str(max_price)
metros = "m2="+"50"

In [13]:
url = "https://www.habitaclia.com/"+property+ascensor+"-"+municipio+".htm?"+habitaciones+baños+maximum_price

In [14]:
# towns = ['barcelona']
'''
'alegria___dulantzi'
 'amurrio',
 'anana',
 'aramaio',
 'arminon',
 'arraia___maeztu',
 'arrazua___ubarrundia']
 '''

"\n'alegria___dulantzi'\n 'amurrio',\n 'anana',\n 'aramaio',\n 'arminon',\n 'arraia___maeztu',\n 'arrazua___ubarrundia']\n "

In [15]:
# Creation of the df to which we will append the properties of the selected towns.
name = []
town = []
area = []
geo_town = []
features = []
m2 = []
n_rooms = []
n_bath = []
price_m2 = []
description = []
price = []
opportunity = []
price_reduction = []
opportunity = []
last_update = []
url = []

x = min(len(name), len(town), len(area), len(description), len(price), len(last_update), len(url))
dct = {'name': name[:x], 'town': town[:x], 'area': area[:x], 'm2': m2[:x], 'n_rooms': n_rooms[:x], 'n_bath': n_bath[:x], 'price_m2': price_m2[:x] ,'price': price[:x], 'price_reduction': price_reduction[:x], 'opportunity':opportunity[:x], 'last_update': last_update[:x],  'description': description[:x], 'url':url[:x] }
df = pd.DataFrame.from_dict(dct)

# Pulling properties from each town in the previously defined list:

for t in towns:
    
    # Getting the number of properties for that town to see how many pages do we need to scrape.
    url = "https://www.habitaclia.com/"+property+"-"+t+".htm?"+maximum_price
    r = requests.get(url)
    r.status_code
    soup = BeautifulSoup(r.content, 'html.parser')
    
    try:
        total_results = int(soup.find('h2', attrs={'class': 'f-right'}).find('span').get_text().replace(".",""))
        pages = range(int(math.floor(total_results/16))+1)
        properties = []

        # Adding all the properties listed in each page to the list.
        for p in pages:
            sleep(randint(2,5))
            url = "https://www.habitaclia.com/"+property+"-"+t+"-"+str(p)+".htm?"+maximum_price
            try:
                r = requests.get(url)
                soup = BeautifulSoup(r.content, 'html.parser')
                properties += soup.find_all('div', attrs={'class': 'list-item-info'})
                del properties[-1] # Last item is an ad
            except:
                print('Error on page', p)
                print('town: ', t, " page: ", p, "url: ", url)

        # Creating a list for each piece of information I want to extract from each property.
        name = []
        town = []
        area = []
        geo_town = []
        features = []
        m2 = []
        n_rooms = []
        n_bath = []
        price_m2 = []
        description = []
        price = []
        opportunity = []
        price_reduction = []
        opportunity = []
        last_update = []
        url = []

        # Getting the information from each property.
        for i,properties in enumerate(properties):

            # Each feature is set as empty prior to being defined. This way we avoid errors when a feature is not available for a certain property.
            name_temp = ""
            town_temp = ""
            area_temp = ""
            geo_town_temp = ""
            m2_temp = ""
            n_rooms_temp = ""
            n_bath_temp = ""
            price_m2_temp = ""
            price_temp = ""
            opportunity_temp = ""
            price_reduction_temp = ""
            description_temp = ""
            last_update_temp = ""
            url_temp = ""

            # other_location enables us to differ between listed properties vs suggested properties, which appear when there are very few properties for one town. We want to avoid them as they are nearby properties not belonging to our target town.
            other_location = properties.find('span', attrs={'class': 'ady-relationship'})
            if other_location is None:
                other_locations_properties = ""
            else: 
                #print(i)
                other_locations_properties = other_location.get_text(strip=True).find('Se encuentra en')

            # Now I am skipping all the properties that are suggested so as to not append them to the df.
            if other_locations_properties == 0:
                pass
            else:
                try:
                    # Extracting the features of a property and saving them in a temporary variable.
                    name_temp = properties.find('h3', attrs={'class': 'list-item-title'}).get_text(strip=True)
                    town_temp = properties.find('p', attrs={'class': 'list-item-location'}).get_text(strip=True).split("-",1)[0].strip().replace("Ver mapa","")
                    area_temp = properties.find('p', attrs={'class': 'list-item-location'}).get_text(strip=True).replace('/','-').strip().replace("Ver mapa","").split("-",1)[0]
                    geo_town_temp = t
                    m2_temp = nfs.get_nums(properties.find('p', attrs={'class': 'list-item-feature'}).get_text(strip=True).split("-")[0])[0] 
                    n_rooms_temp = nfs.get_nums(properties.find('p', attrs={'class': 'list-item-feature'}).get_text(strip=True).split("-")[1])[0]
                    n_bath_temp = nfs.get_nums(properties.find('p', attrs={'class': 'list-item-feature'}).get_text(strip=True).split("-")[2])[0] 
                    price_m2_temp = nfs.get_nums(properties.find('p', attrs={'class': 'list-item-feature'}).get_text(strip=True).split("-")[3])[0] 
                    raw_price_temp = properties.find('article', attrs={'class': 'list-item-price'}).get_text()
                    if search("Oportunidad", raw_price_temp):
                        if search("ha bajado", raw_price_temp): 
                            price_temp = nfs.get_nums((properties.find('article', attrs={'class': 'list-item-price'}).get_text()).replace(".",""))[0]
                            price_reduction_temp = nfs.get_nums((properties.find('article', attrs={'class': 'list-item-price'}).get_text()).replace(".",""))[1]
                            opportunity_temp = "yes"
                        else:
                            price_temp = nfs.get_nums((properties.find('article', attrs={'class': 'list-item-price'}).get_text()).replace(".",""))[0]
                            price_reduction_temp = "0"
                            opportunity_temp = "yes"
                    elif search("ha bajado", raw_price_temp):
                        price_temp = nfs.get_nums((properties.find('article', attrs={'class': 'list-item-price'}).get_text()).replace(".",""))[0]
                        price_reduction_temp = nfs.get_nums((properties.find('article', attrs={'class': 'list-item-price'}).get_text()).replace(".",""))[1]      
                        opportunity_temp = "no"
                    else: 
                        price_temp = nfs.get_nums((properties.find('article', attrs={'class': 'list-item-price'}).get_text()).replace(".",""))[0]
                        price_reduction_temp = "0"
                        opportunity_temp = "no"
                    description_temp = properties.find('p', attrs={'class': 'list-item-description'}).get_text(strip=True)
                    last_update_temp = nfs.get_nums(properties.find('span', attrs={'class': 'list-item-date'}).get_text(strip=True))
                    url_temp = properties.find('h3', attrs={'class': 'list-item-title'}).find('a').get('href')

                    # Appending temporary variables features to their corresponding list.
                    name.append(name_temp)
                    town.append(town_temp)
                    area.append(area_temp)
                    geo_town.append(geo_town_temp)
                    m2.append(m2_temp)
                    n_rooms.append(n_rooms_temp)
                    n_bath.append(n_bath_temp)
                    price_m2.append(price_m2_temp)
                    price.append(price_temp)
                    opportunity.append(opportunity_temp)
                    price_reduction.append(price_reduction_temp)
                    description.append(description_temp)
                    last_update.append(last_update_temp)
                    url.append(url_temp)

                except:
                    # In case we may encounter an error, we print the features of each property to find the bug.
                    '''
                    print('------------------------------------')
                    print('nombre: ', name_temp)
                    print('town_temp: ', town_temp)
                    print('area_temp: ', area_temp)
                    print('m2_temp: ', m2_temp)
                    print('n_rooms_temp: ', n_rooms_temp)
                    print('n_bath_temp: ', n_bath_temp)
                    print('price_m2_temp: ', price_m2_temp)
                    print('price_temp: ', price_temp)
                    print('opportunity_temp: ', opportunity_temp)
                    print('price_reduction_temp: ', price_reduction_temp)
                    print('description_temp: ', description_temp)
                    print('last_update_temp: ', last_update_temp)
                    print('url_temp: ', url_temp)
                    print('------------------------------------')
                    '''

        x_town = min(len(name), len(town), len(area), len(geo_town), len(description), len(price), len(last_update), len(url))
        dct_town = {'name': name[:x_town], 'town': town[:x_town], 'area': area[:x_town], 'geo_town':geo_town[:x_town],'m2': m2[:x_town], 'n_rooms': n_rooms[:x_town], 'n_bath': n_bath[:x_town], 'price_m2': price_m2[:x_town] ,'price': price[:x_town], 'price_reduction': price_reduction[:x_town], 'opportunity':opportunity[:x_town], 'last_update': last_update[:x_town],  'description': description[:x_town], 'url':url[:x_town] }
        df_town = pd.DataFrame.from_dict(dct_town)
        df = df.append(df_town, ignore_index = True)
        
        #Cleaning the final df
        #df= df[df['price']<int(max_price)]
        #df = df[~df['description'].str.contains('nuda|sin cedula|sin cédula')]

    except:
        print("no properties found at: ", url)

https://www.habitaclia.com/viviendas-badalona-0.htm?pmax=90000
town:  badalona  page:  0
https://www.habitaclia.com/viviendas-badalona-1.htm?pmax=90000
town:  badalona  page:  1
https://www.habitaclia.com/viviendas-badalona-2.htm?pmax=90000
town:  badalona  page:  2
https://www.habitaclia.com/viviendas-badalona-3.htm?pmax=90000
town:  badalona  page:  3
https://www.habitaclia.com/viviendas-badalona-4.htm?pmax=90000
town:  badalona  page:  4
https://www.habitaclia.com/viviendas-badalona-5.htm?pmax=90000
town:  badalona  page:  5
https://www.habitaclia.com/viviendas-badalona-6.htm?pmax=90000
town:  badalona  page:  6
https://www.habitaclia.com/viviendas-badalona-7.htm?pmax=90000
town:  badalona  page:  7
https://www.habitaclia.com/viviendas-badalona-8.htm?pmax=90000
town:  badalona  page:  8
https://www.habitaclia.com/viviendas-badalona-9.htm?pmax=90000
town:  badalona  page:  9
https://www.habitaclia.com/viviendas-badalona-10.htm?pmax=90000
town:  badalona  page:  10
https://www.habitac

town:  manresa  page:  9
https://www.habitaclia.com/viviendas-manresa-10.htm?pmax=90000
town:  manresa  page:  10
https://www.habitaclia.com/viviendas-manresa-11.htm?pmax=90000
town:  manresa  page:  11
https://www.habitaclia.com/viviendas-manresa-12.htm?pmax=90000
town:  manresa  page:  12
https://www.habitaclia.com/viviendas-manresa-13.htm?pmax=90000
town:  manresa  page:  13
https://www.habitaclia.com/viviendas-manresa-14.htm?pmax=90000
town:  manresa  page:  14
https://www.habitaclia.com/viviendas-manresa-15.htm?pmax=90000
town:  manresa  page:  15
https://www.habitaclia.com/viviendas-manresa-16.htm?pmax=90000
town:  manresa  page:  16
https://www.habitaclia.com/viviendas-manresa-17.htm?pmax=90000
town:  manresa  page:  17
https://www.habitaclia.com/viviendas-manresa-18.htm?pmax=90000
town:  manresa  page:  18
https://www.habitaclia.com/viviendas-manresa-19.htm?pmax=90000
town:  manresa  page:  19
https://www.habitaclia.com/viviendas-manresa-20.htm?pmax=90000
town:  manresa  page: 

town:  terrassa  page:  15
https://www.habitaclia.com/viviendas-terrassa-16.htm?pmax=90000
town:  terrassa  page:  16
https://www.habitaclia.com/viviendas-terrassa-17.htm?pmax=90000
town:  terrassa  page:  17
https://www.habitaclia.com/viviendas-terrassa-18.htm?pmax=90000
town:  terrassa  page:  18
https://www.habitaclia.com/viviendas-terrassa-19.htm?pmax=90000
town:  terrassa  page:  19
https://www.habitaclia.com/viviendas-terrassa-20.htm?pmax=90000
town:  terrassa  page:  20
https://www.habitaclia.com/viviendas-tordera-0.htm?pmax=90000
town:  tordera  page:  0
https://www.habitaclia.com/viviendas-tordera-1.htm?pmax=90000
town:  tordera  page:  1
https://www.habitaclia.com/viviendas-tordera-2.htm?pmax=90000
town:  tordera  page:  2
https://www.habitaclia.com/viviendas-tordera-3.htm?pmax=90000
town:  tordera  page:  3
https://www.habitaclia.com/viviendas-tordera-4.htm?pmax=90000
town:  tordera  page:  4
https://www.habitaclia.com/viviendas-vic-0.htm?pmax=90000
town:  vic  page:  0
http

https://www.habitaclia.com/viviendas-salou-1.htm?pmax=90000
town:  salou  page:  1
https://www.habitaclia.com/viviendas-salou-2.htm?pmax=90000
town:  salou  page:  2
https://www.habitaclia.com/viviendas-salou-3.htm?pmax=90000
town:  salou  page:  3
https://www.habitaclia.com/viviendas-salou-4.htm?pmax=90000
town:  salou  page:  4
https://www.habitaclia.com/viviendas-salou-5.htm?pmax=90000
town:  salou  page:  5
https://www.habitaclia.com/viviendas-salou-6.htm?pmax=90000
town:  salou  page:  6
https://www.habitaclia.com/viviendas-salou-7.htm?pmax=90000
town:  salou  page:  7
https://www.habitaclia.com/viviendas-salou-8.htm?pmax=90000
town:  salou  page:  8
https://www.habitaclia.com/viviendas-salou-9.htm?pmax=90000
town:  salou  page:  9
https://www.habitaclia.com/viviendas-salou-10.htm?pmax=90000
town:  salou  page:  10
https://www.habitaclia.com/viviendas-salou-11.htm?pmax=90000
town:  salou  page:  11
https://www.habitaclia.com/viviendas-salou-12.htm?pmax=90000
town:  salou  page:  1

town:  tarragona  page:  10
https://www.habitaclia.com/viviendas-tarragona-11.htm?pmax=90000
town:  tarragona  page:  11
https://www.habitaclia.com/viviendas-tarragona-12.htm?pmax=90000
town:  tarragona  page:  12
https://www.habitaclia.com/viviendas-tarragona-13.htm?pmax=90000
town:  tarragona  page:  13
https://www.habitaclia.com/viviendas-tarragona-14.htm?pmax=90000
town:  tarragona  page:  14
https://www.habitaclia.com/viviendas-tarragona-15.htm?pmax=90000
town:  tarragona  page:  15
https://www.habitaclia.com/viviendas-tarragona-16.htm?pmax=90000
town:  tarragona  page:  16
https://www.habitaclia.com/viviendas-tarragona-17.htm?pmax=90000
town:  tarragona  page:  17
https://www.habitaclia.com/viviendas-tarragona-18.htm?pmax=90000
town:  tarragona  page:  18
https://www.habitaclia.com/viviendas-tarragona-19.htm?pmax=90000
town:  tarragona  page:  19
https://www.habitaclia.com/viviendas-tarragona-20.htm?pmax=90000
town:  tarragona  page:  20
https://www.habitaclia.com/viviendas-tarra

In [66]:
df.shape

(6241, 15)

In [67]:
df1 = df.copy()

In [68]:
df1 = df1.merge(geo_data, left_on='geo_town', right_on='town', how='left')
df1 = df1.drop(["town_x", "town_y", "n_properties"], axis=1)
df1 = df1.loc[:, ~df1.columns.str.contains('^Unnamed')]

In [69]:
numericals = ['m2', 'n_rooms', 'n_bath', 'price_m2', 'price', 'price_reduction', 'last_update']
def anytype_to_numerical(df, columns = []):
    for c in columns:
        if df[c].dtypes == 'float64':
            df[c] = df[c].astype(int)
anytype_to_numerical(df1, columns = numericals)

def clean_last_update():
    for i,n in enumerate(df1['last_update']):
        try:
            df1['last_update'][i] = df1['last_update'][i][0]
        except:
            df1['last_update'][i] = '0'
clean_last_update()

<ipython-input-69-0fb72c807b2d>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['last_update'][i] = df1['last_update'][i][0]
<ipython-input-69-0fb72c807b2d>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['last_update'][i] = '0'


In [74]:
df1 = df1.drop('piso', axis=1)

In [75]:
df1.head(3)

,name,area,m2,n_rooms,n_bath,price_m2,price,price_reduction,opportunity,last_update,description,url,geo_town,province,ccaa
0,Piso Carrer mina. Fabuloso piso luminoso,Badalona,55,3,1,1,97000,18000,yes,2,"Piso de 55m², 1º de altura real, se compone de...",https://www.habitaclia.com/comprar-piso-fabulo...,badalona,barcelona,cataluña
1,Piso Carrer mina. Oportunidad por precio y zona,Badalona,55,2,1,1,97000,0,no,0,"Piso de 55m², 2º de altura real, se compone de...",https://www.habitaclia.com/comprar-piso-oportu...,badalona,barcelona,cataluña
2,Piso Avinguda marquès de montroig. Piso con 3...,Badalona,71,3,1,1,97260,0,yes,0,¡¡¡OPORTUNIDAD POR ESTADO Y PRECIO!!! Vivienda...,https://www.habitaclia.com/comprar-piso-con_3_...,badalona,barcelona,cataluña


In [76]:
today = datetime.now().strftime('%Y.%m.%d')
df1.to_csv(path_or_buf = '/Users/ignaciolorenzoqueralt/Documents/Ironhack/Final Project/properties/sale/'+today+'_'+ccaa+'_'+str(max_price)+'_'+str(num_properties_per_town)+'.csv')